In [1]:
import models
from torchvision import transforms
import data_loader
import lib_generation
import torch
from torch.autograd import Variable
import numpy as np


model = models.ResNet34(num_c=10)
model.load_state_dict(torch.load("pre_trained/resnet_cifar10.pth", map_location = "cuda:" + str(0)))
in_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

train_loader, test_loader = data_loader.getTargetDataSet('cifar10', 16, in_transform, './data')

model.cuda()
# set information about feature extaction
model.eval()
temp_x = torch.rand(2,3,32,32).cuda()
temp_x = Variable(temp_x)
temp_list = model.feature_list(temp_x)[1]
num_output = len(temp_list)
feature_list = np.empty(num_output)
count = 0
for out in temp_list:
    feature_list[count] = out.size(1)
    count += 1

sample_mean, precision = lib_generation.sample_estimator(model, 10, feature_list, train_loader)

outf = './output/resnet_cifar10/'

/home/zjiaming/anaconda3/envs/randomTexture/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


/home/zjiaming/deep_Mahalanobis_detector/lib_generation.py:69: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data = Variable(data, volatile=True)



 Training Accuracy:(99.75%)



In [3]:
class AC(torch.nn.Module):
    def __init__(self, model, num_classes, outf, out_flag, net_type, sample_mean, precision, layer_index, magnitude, confidence_mean, confidence_std):
        super(AC, self).__init__()

        self.model = model
        self.layer_index = layer_index
        self.sample_mean = sample_mean
        self.precision = precision
        self.num_classes = num_classes
        self.layers = layer_index
        self.confidence_mean = confidence_mean
        self.confidence_std = confidence_std

        self.linear = torch.nn.Linear(5, 1).cuda()

    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    def forward(self, x):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        confidence_score = 0
        for layers in range(self.layers):
            out_features = self.model.intermediate_forward(x, layers)
            out_features = out_features.view(out_features.size(0), out_features.size(1), -1)
            out_features = torch.mean(out_features, 2)

            # compute Mahalanobis score
            gaussian_score = 0
            for i in range(self.num_classes):
                batch_sample_mean = self.sample_mean[layers][i]
                # zero_f = out_features.data - batch_sample_mean
                zero_f = out_features - batch_sample_mean
                term_gau = -0.5*torch.mm(torch.mm(zero_f, self.precision[layers]), zero_f.t()).diag()
                if i == 0:
                    gaussian_score = term_gau.view(-1,1)
                else:
                    gaussian_score = torch.cat((gaussian_score, term_gau.view(-1,1)), 1)
            
            gaussian_score, _ = torch.torch.max(gaussian_score, dim=1)
            
            gaussian_score = torch.unsqueeze(gaussian_score, dim=1)
            #normalize
            gaussian_score = (gaussian_score - self.confidence_mean[layers]) / self.confidence_std[layers]
            if layers == 0:
                confidence_score = gaussian_score
            else:
                confidence_score = torch.cat((confidence_score, gaussian_score), 1)

        outputs = torch.sigmoid(self.linear(confidence_score))
        return outputs


In [4]:
def get_Mahalanobis_score(model, test_loader, num_classes, outf, out_flag, net_type, sample_mean, precision, layer_index, magnitude):
    '''
    Compute the proposed Mahalanobis confidence score on input dataset
    return: Mahalanobis score from layer_index
    '''
    model.eval()
    # Mahalanobis = torch.tensor([1])
    Mahalanobis = []

    if out_flag:
        filename = 'output/{}/{}_layer{}.txt'.format(outf, net_type, layer_index)
        if not os.path.exists('output/{}'.format(outf)):
            os.mkdir('output/{}'.format(outf))
        g = open(filename, 'w+')

    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, requires_grad = True), Variable(target)
        
        out_features = model.intermediate_forward(data, layer_index)
        
        out_features = out_features.view(out_features.size(0), out_features.size(1), -1)
        out_features = torch.mean(out_features, 2)

        # compute Mahalanobis score
        gaussian_score = 0
        for i in range(num_classes):
            batch_sample_mean = sample_mean[layer_index][i]
            # zero_f = out_features.data - batch_sample_mean
            zero_f = out_features - batch_sample_mean
            term_gau = -0.5*torch.mm(torch.mm(zero_f, precision[layer_index]), zero_f.t()).diag()
            if i == 0:
                gaussian_score = term_gau.view(-1,1)
            else:
                gaussian_score = torch.cat((gaussian_score, term_gau.view(-1,1)), 1)
        
        gaussian_score, _ = torch.max(gaussian_score, dim=1)
        Mahalanobis.extend(gaussian_score.cpu().detach().numpy())
        # Mahalanobis = torch.cat((Mahalanobis, gaussian_score.cpu()))
        if out_flag:
            for i in range(data.size(0)):
                g.write("{}\n".format(gaussian_score[i]))
                
    return Mahalanobis

In [6]:
# get dataset
in_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

cifar_train_loader, cifar_test_loader = data_loader.getTargetDataSet('cifar10', 16, in_transform, 'output/resnet_cifar10/')

svhn_train_loader, svhn_test_loader = data_loader.getSVHN(batch_size=16, TF=in_transform, data_root='data/', num_workers=1)


# std_weight = torch.tensor([0.03915799,  0.02396634, -0.09159853, -0.00617044, -0.00793493], requires_grad=True)
# std_bias = torch.tensor([-11.964], requires_grad=True)
# with torch.no_grad():
#     print(d_model.linear.weight)
#     d_model.linear.weight[0] = std_weight
#     d_model.linear.bias[0] = std_bias


Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: data/svhn-data/train_32x32.mat
Using downloaded and verified file: data/svhn-data/test_32x32.mat


In [7]:
# Get all the training data confidence score
outf = 'cifar_resnet_paper'
layer_means = []
layer_stds = []
for i in range(len(model.feature_list(temp_x)[1])):
    M_in = get_Mahalanobis_score(model, train_loader, 10, outf, \
                    False, 'resnet_paper', sample_mean, precision, i, 0.0)
    layer_means.append(np.mean(M_in))
    layer_stds.append(np.std(M_in))

In [8]:
d_model = AC(model, 10, outf, True, 'resnet', sample_mean, precision, 5, 0.0, layer_means, layer_stds)

In [22]:
layer_means

[-10.736551, -31.203535, -63.392277, -127.96577, -255.86098]

# Training the Model we use to reduce OOD

In [12]:
# Fix the top layers
for i, p in enumerate(d_model.parameters()):
    if i < 110:
        p.requires_grad = False

In [28]:
import tqdm
learning_rate = 0.1
criterion = torch.nn.BCELoss()
# criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(d_model.parameters(), lr=learning_rate)

for epoch in range(10):
    i = 0
    losses = 0
    for data, data2 in zip(cifar_train_loader, svhn_train_loader):
        data, labels = data[0].cuda(), data[1].cuda()
        data, labels = Variable(data, requires_grad = True), Variable(labels)
        labels = torch.ones(labels.size()[0]).cuda()

        out_data, out_labels = data2[0].cuda(), data2[1].cuda()
        out_data, out_labels = Variable(out_data, requires_grad = True), Variable(out_labels)
        out_labels = torch.zeros(out_labels.size()[0]).cuda()
        # print(data)
        data = torch.concat((data, out_data), 0)
        labels = torch.concat((labels,out_labels), 0)
        optimizer.zero_grad()
        outputs = d_model(data)
        outputs = torch.squeeze(outputs)
        print(outputs, labels)
        # outputs = outputs.view(outputs.size(0))
        # print(outputs.shape)
        loss = criterion(outputs, labels.type(torch.float))
        # print(loss)
        # break
        # loss = Variable(loss, requires_grad=True)
        loss.backward()
        
        optimizer.step()
        i += 1
        losses += loss.item()
        if i % 100 == 0:
            print(loss.item())
    print(losses)
    


tensor([9.9141e-01, 9.8347e-01, 9.7844e-01, 9.5914e-01, 7.4600e-01, 9.9631e-01,
        9.2015e-01, 9.7368e-01, 9.9545e-01, 9.9447e-01, 9.8544e-01, 9.2207e-01,
        9.4190e-01, 9.9210e-01, 9.9518e-01, 9.9790e-01, 7.9996e-02, 1.4813e-03,
        7.6268e-05, 2.5515e-05, 4.4268e-03, 2.0032e-04, 1.2437e-05, 1.2815e-03,
        6.0942e-02, 3.7039e-08, 2.6359e-04, 2.3725e-02, 1.8609e-04, 1.1843e-01,
        1.3879e-04, 1.4204e-01], device='cuda:0', grad_fn=<SqueezeBackward0>) tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
tensor([9.8955e-01, 9.9499e-01, 9.9704e-01, 9.7758e-01, 9.9478e-01, 9.9751e-01,
        9.6681e-01, 9.6464e-01, 9.9736e-01, 9.9985e-01, 9.8422e-01, 9.9670e-01,
        9.6011e-01, 2.2550e-01, 9.9089e-01, 9.8808e-01, 6.5908e-01, 6.1427e-03,
        2.3033e-01, 3.6699e-01, 1.6310e-02, 1.7165e-08, 1.2350e-02, 3.7166e-03,
        1.3913e-06, 1.8486e-04, 2

KeyboardInterrupt: 

In [25]:
for p in d_model.parameters():
    print(p)

Parameter containing:
tensor([[[[-4.2463e-03, -5.5520e-02, -3.7956e-02],
          [-2.5731e-02, -1.4475e-01, -1.0340e-01],
          [-1.3080e-02, -3.0984e-02,  8.7611e-03]],

         [[ 1.2846e-02, -4.5997e-02, -3.4822e-02],
          [ 4.3854e-03, -1.1815e-01, -8.9485e-02],
          [ 1.7848e-02, -7.6388e-03,  2.4630e-02]],

         [[ 4.1093e-02,  1.1733e-02,  2.2779e-02],
          [ 4.7384e-02, -4.2903e-02, -1.9005e-02],
          [ 6.3438e-02,  5.2970e-02,  7.8736e-02]]],


        [[[ 1.5232e-01, -9.8197e-02, -6.8447e-02],
          [ 1.0159e-01,  2.0529e-01, -2.6144e-01],
          [ 1.5096e-01, -5.2695e-02, -1.4868e-01]],

         [[ 7.6155e-03, -5.0022e-02,  3.6474e-02],
          [-1.3100e-01,  3.4280e-01,  5.1537e-02],
          [-6.5635e-02, -8.7332e-02, -1.2540e-02]],

         [[-4.2815e-02, -9.2617e-02,  6.3412e-02],
          [-2.2528e-01,  1.6167e-01,  1.5167e-01],
          [-1.1018e-01, -3.0992e-02,  8.9563e-02]]],


        [[[-1.3613e-06, -1.1690e-06, -1.5523

In [26]:
layer_means
layer_stds

[14.008426, 21.433807, 23.49635, 31.416542, 130.22836]

In [27]:
layer_means

[-10.736551, -31.203535, -63.392277, -127.96577, -255.86098]

# Reducing OODness

In [ ]:
# First turn the layer back on
# Fix the top layers
for i, p in enumerate(d_model.parameters()):
    p.requires_grad = True

In [30]:
# Get the model started
from attacks import ood_pgd
attack = ood_pgd
ce_loss = torch.nn.BCELoss()
soft = torch.nn.Softmax(dim=1)

model.eval()
d_model.eval()
with torch.no_grad():
    for data, target in cifar_test_loader:
        data, labels = data.cuda(), data.cuda()
        data, labels = Variable(data, requires_grad = True), Variable(labels)

        # zero meaning ood examples, we want to stay away
        d_labels = torch.zeros(labels.size()[0]).cuda()
        
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        # print(ce_loss(d_model(data), d_target))
        print("before", d_model(data))
        print("before confidence", soft(outputs.data))
        
        with torch.enable_grad():
            reduce_ood_inputs = attack.perturb(d_model, data, data.clone(), d_labels, num_steps=10, attack=True, step_size=0.07, epsilon=0.031, celoss=False)

        print("after", d_model(reduce_ood_inputs))
        print("reduce ood", soft(model(reduce_ood_inputs).data))

before tensor([[0.7128],
        [0.9691],
        [0.8993],
        [0.3297],
        [0.9400],
        [0.9723],
        [0.5171],
        [0.9867],
        [0.9963],
        [0.9984],
        [0.9957],
        [0.9950],
        [0.9823],
        [0.9944],
        [0.9965],
        [0.3159]], device='cuda:0')
before confidence tensor([[3.0283e-05, 4.2229e-05, 1.0920e-05, 9.9961e-01, 1.4067e-05, 1.3729e-04,
         1.2954e-04, 1.1758e-05, 7.7413e-06, 5.9452e-06],
        [2.0396e-05, 6.4483e-05, 5.0365e-06, 9.4132e-06, 4.3504e-06, 6.7227e-06,
         1.1908e-05, 5.6047e-06, 9.9987e-01, 6.0123e-06],
        [1.4476e-03, 7.5674e-01, 6.0230e-04, 3.8191e-04, 4.5819e-04, 5.7109e-04,
         6.9231e-04, 5.3860e-04, 2.3446e-01, 4.1062e-03],
        [9.8827e-01, 3.2023e-04, 5.1467e-03, 5.0750e-04, 1.5264e-04, 2.0724e-04,
         3.4502e-04, 3.3499e-04, 5.4772e-04, 4.1680e-03],
        [2.4646e-05, 7.7813e-05, 1.8792e-04, 6.1568e-05, 7.3602e-05, 2.7898e-05,
         9.9947e-01, 1.7561e-05,

ValueError: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.

In [120]:
# Get all the training data confidence score
outf = 'svhn_resnet_paper'
svhn_layer_means = []
svhn_layer_stds = []
for i in range(len(model.feature_list(temp_x)[1])):
    M_in = get_Mahalanobis_score(model, svhn_train_loader, 10, outf, \
                    True, 'resnet_paper', sample_mean, precision, i, 0.0)
    svhn_layer_means.append(np.mean(M_in))
    svhn_layer_stds.append(np.std(M_in))

In [112]:
from torch.utils.data import Dataset, DataLoader

class ConfidenceDataset(Dataset):
    def __init__(self, confidence, labels):
        self.labels = labels
        self.confidence = confidence

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        label = self.labels[idx]
        conf = self.confidence[idx]
        sample = {"Confidence": conf, "Class": label}
        return sample

In [274]:
X_train = torch.zeros([1000, 5], dtype=torch.float)
Y_train = torch.zeros([1000, 1], dtype=torch.float)
# X_train = np.zeros((1000, 5))
# Y_train = np.zeros((1000, 1))
X_test = torch.zeros([1000, 5], dtype=torch.float)
Y_test = torch.zeros([1000, 1], dtype=torch.float)
for j in range(5):
    with open("output/cifar_resnet_paper/resnet_paper_layer{}.txt".format(j)) as myfile:
        head = [next(myfile) for x in range(1000)]
    with open("output/svhn_resnet_paper/resnet_paper_layer{}.txt".format(j)) as myfile:
        svhn_head = [next(myfile) for x in range(1000)]
    for i in range(500):
        X_train[i][j] = (float(head[i]) - layer_means[j]) / layer_stds[j]
        X_train[i + 500][j] = (float(svhn_head[i]) - layer_means[j]) / layer_stds[j]
        Y_train[i] = float(1)
        Y_train[i + 500] = float(0)

    # for i in range(500, 1000):
    #     X_test[i - 500][j] = float(head[i])
    #     Y_test[i - 500][j] = float(svhn_head[i])
X_train

tensor([[-0.1857,  0.2076,  0.4815, -0.4128,  0.8000],
        [ 0.3774,  0.4379,  0.1944,  0.1607, -3.0924],
        [ 0.0581, -1.2707, -0.3739, -0.4778,  0.1568],
        ...,
        [-0.1355, -2.3011, -2.2267, -3.3557, -7.0039],
        [ 0.1520, -1.3678, -6.9439, -1.6912, -2.9958],
        [-0.3230, -1.8619, -2.9302, -6.9449, -3.5742]])

In [276]:
confidence_train_dataset = ConfidenceDataset(X_train, Y_train)
# confidence_test_dataset = ConfidenceDataset(X_train, Y_train)
confidence_train_loader = torch.utils.data.DataLoader(confidence_train_dataset, batch_size=16,
                        shuffle=True, num_workers=1)

In [283]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
         
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

In [127]:
confidence_train_dataset[0]

{'Confidence': tensor([ -13.3377,  -26.7544,  -52.0791, -140.9334, -151.6730]),
 'Class': tensor([1.])}

In [124]:
s = 0
for i in range(1000):
    if y_pred.round()[i] == Y_train[i]:
        s += 1
s

985

In [292]:
lr.coef_

array([[0.21226137, 1.25658983, 0.69801996, 1.28676522, 1.57649249]])

In [291]:
lr.intercept_

array([6.4821957])

In [278]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0).fit(X_train, Y_train)
y_pred = lr.predict_proba(X_train)[:, 1]
y_pred.round()

/home/zjiaming/anaconda3/envs/randomTexture/lib/python3.10/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [174]:
m = torch.nn.Sigmoid()
loss = torch.nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)
output2 = loss(torch.unsqueeze(m(input), dim=0), torch.unsqueeze(target, dim=0))
print(output, output2)

tensor(1.2052, grad_fn=<BinaryCrossEntropyBackward0>) tensor(1.2052, grad_fn=<BinaryCrossEntropyBackward0>)


In [297]:
logistic_model = LogisticRegression(5, 1)
logistic_model.cuda()
logistic_model.train()
# criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=logistic_model.parameters(), lr=0.1)


In [260]:
criterion = torch.nn.BCELoss()

In [304]:
lr.coef_

array([[0.21226137, 1.25658983, 0.69801996, 1.28676522, 1.57649249]])

In [303]:
for para in list(logistic_model.parameters()):
    print(para)

Parameter containing:
tensor([[0.2755, 1.4022, 0.7254, 1.4311, 1.7225]], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([7.0921], device='cuda:0', requires_grad=True)


In [302]:
# criterion = torch.nn.CrossEntropyLoss()  
with torch.enable_grad():
    for i in range(1):
        losses = 0.0
        for j, data_batch in enumerate(confidence_train_loader):
            # if j == 0:
            #     print(j, data_batch['Confidence'], data_batch['Class'])
            data, target = data_batch['Confidence'].cuda(), data_batch['Class'].cuda()
            data, target = Variable(data, requires_grad=True), Variable(target)

            # target = torch.unsqueeze(torch.zeros(16), dim=1).cuda()
            # print(torch.unsqueeze(outputs), torch.unsqueeze(target), criterion(outputs, target))
            # break

            # logistic_model.train()

            optimizer.zero_grad()
            outputs = logistic_model(data)
            # if j == 0:
            print( outputs.round(), target)
            loss = criterion(outputs, target)
            # print("gradient")
            # loss.register_hook(lambda grad: print(grad))
            loss.backward()
            # print("loss", loss)
            optimizer.step()
            # print(logistic_model.linear.weight.grad)
            # print(optimizer.state)
            losses += loss.item()
            # if j == 2:
            #     break
        # break
        # for para in logistic_model.parameters():
        #     print(para)
        print(losses)


tensor([[1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.]], device='cuda:0', grad_fn=<RoundBackward0>) tensor([[1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.]], device='cuda:0')
tensor([[1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]], device='cuda:0', grad_fn=<RoundBackward0>) tensor([[1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]

In [ ]:
model()

In [134]:
for para in logistic_model.parameters():
    print(para)

Parameter containing:
tensor([[-0.3597, -0.0205, -0.1117, -0.0753,  0.2670]], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([0.2909], device='cuda:0', requires_grad=True)


In [36]:
len(M_in)

50000

Record All the confidence score by zjiaming


Testing


In [ ]:
d_model.eval()

model.cuda()
d_model.cuda()
from attacks import ood_pgd
attack = ood_pgd
ce_loss = torch.nn.BCELoss()
soft = torch.nn.Softmax(dim=1)

model.eval()
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, requires_grad = True), Variable(target)

        d_target = torch.unsqueeze(torch.ones(16), dim=1).cuda()
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        # print(ce_loss(d_model(data), d_target))
        print("before", d_model(data))
        print("before confidence", soft(outputs.data))
        # print(outputs)
        # print(d_model(data))
        with torch.enable_grad():
            attack_inputs = attack.perturb(model, data, data.clone(), target, num_steps=10, attack=True, step_size=0.07, epsilon=0.031, celoss=True)
        print("after attack", soft(model(attack_inputs).data))
        with torch.enable_grad():
            inputs = attack.perturb(d_model, attack_inputs, attack_inputs.clone(), d_target, num_steps=20, attack=True, step_size=0.07, epsilon=0.031, celoss=False)
        # with torch.enable_grad():
        #     inputs = attack.perturb(model, data, data.clone(), target, num_steps=1, attack=True, epsilon=0.0031)
        print("after", d_model(inputs))
        print("reduce ood", soft(model(inputs).data))
        break
    
